In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
#added correct information
username = "aacuser"
password = "CS340PASS"
HOST = "nv-desktop-services.apporto.com"
PORT = 31990
DB = "aac"
COL = "animals"
shelter = AnimalShelter(username, password, HOST, PORT, DB, COL)

# Connect to database via CRUD Module commented out since I do it above
#db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Grazioso Salvare - Zachariah Garrison') #added name 

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # added image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    #added image, clickable to site
    html.A(
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style = {'width': '150px'}),
        href = 'https://www.snhu.edu',
        target = '_blank'
    ),
    #Site title
    html.Center(html.B(html.H1('Grazioso Salvare - Zachariah Garrison'))),
    html.Hr(),
    #add radio buttons to filter rescue breeds
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.RadioItems(
        id ='filter-type',
        options =[
            {'label': 'Water Rescue', 'value': 'WATER'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER'},
            {'label': 'Reset', 'value': 'RESET'}
        ],
        value = 'RESET', #add reset button 
        labelStyle = {'display': 'inline-block', 'margin-right': '15px'},
        style = {'margin-top': '10px'}
        )          
    ]),
    html.Hr(),
    #display animal data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10, #Limits number of rows
        row_selectable="single", #Only select one row
        selected_rows = [],
        filter_action="native", #allow filtering
        sort_action="native", #allow sorting
        page_action="native", 
        page_current=0, #Set Page View
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'}, #apply to odd rows
                'backgroundColor': 'rgb(220, 220, 220)', #add grey
            },            
            {
                'if':{'state': 'selected'}, #add selected row highlight
                'backgroundColor': 'rgb(173, 216, 230)', #add blue
                'border': '2px solid blue',
            },
        ],        
        style_header={
            'backgroundColor': 'grey', #add grey highlight to header
            'color': 'white', #put white font
            'fontWeight': 'bold', #add bold font
        },
                            
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################   
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #Filter Rescue Types
    if filter_type == 'WATER':
        filtered_data = shelter.get_water_rescue_dogs()
    elif filter_type == 'MOUNTAIN':
        filtered_data = shelter.get_mountain_rescue_dogs()
    elif filter_type == 'DISASTER':
        filtered_data = shelter.get_disaster_rescue_dogs()
    else:  # RESET
        filtered_data = shelter.get_all_dogs()
    
    df_filtered = pd.DataFrame.from_records(filtered_data)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #Update chart based on data selected
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart")]
    dff = pd.DataFrame.from_dict(viewData)
    
    # Count breed numbers and get top 10 breeds
    breed_counts = dff['breed'].value_counts().nlargest(10)
    other_count = dff['breed'].value_counts().iloc[10:].sum()
    
    # Create a new DataFrame for chart
    plot_data = pd.concat([breed_counts, pd.Series({'Other': other_count}) if other_count > 0 else pd.Series()])
    
    plot_data = plot_data.reset_index()
    plot_data.columns = ['breed', 'count']
    
    # Create pie chart 
    fig = px.pie(
        plot_data.reset_index(),
        names='breed',
        values='count',
        title='Breed Distribution',
        hole=0.3
    )
    
    return [dcc.Graph(figure=fig)]

#This callback will highlight a cell on the data table when the user selects it
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    base_style = [
        {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(220, 220, 220)'},
        {'if': {'state': 'selected'}, 'backgroundColor': 'rgb(173, 216, 230)'}
    ]
    
    if not selected_columns:  # Handles None or empty list
        return base_style
    
    column_styles = [
        {'if': {'column_id': col}, 'backgroundColor': '#D2F3FF'}
        for col in selected_columns
    ]
    
    return base_style + column_styles


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0: #Check all cases, was getting error
        return [] #No map if no data 
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0] #adjusted from check above 
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)
